In [ ]:
sfrom google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Run below commands in google colab
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.0
#!wget -q http://apache.osuosl.org/spark/spark-3.4.2/spark-3.4.2-bin-hadoop3.tgz
# unzip it
!tar xf /content/drive/MyDrive/spark-3.4.2-bin-hadoop3.tgz
# install findspark
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.2-bin-hadoop3"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate()

assert  "3." in sc.version, "Verify that the cluster Spark's version is 3.x"

In [ ]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

fine init

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window
from pyspark.sql.functions import col, count

In [ ]:
spark = SparkSession(sc)
print(spark)

In [ ]:

avocado_path = "/content/drive/MyDrive/avocado.csv"
df_avocado = spark.read \
                    .option("inferSchema", "true") \
                    .option("header", "true") \
                    .csv(avocado_path)

df_avocado.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- AveragePrice: double (nullable = true)
 |-- Total Volume: double (nullable = true)
 |-- 4046: double (nullable = true)
 |-- 4225: double (nullable = true)
 |-- 4770: double (nullable = true)
 |-- Total Bags: double (nullable = true)
 |-- Small Bags: double (nullable = true)
 |-- Large Bags: double (nullable = true)
 |-- XLarge Bags: double (nullable = true)
 |-- type: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- region: string (nullable = true)



In [ ]:
df_avocado.show(10)

+---+----------+------------+------------+-------+---------+-----+----------+----------+----------+-----------+------------+----+------+
|_c0|      Date|AveragePrice|Total Volume|   4046|     4225| 4770|Total Bags|Small Bags|Large Bags|XLarge Bags|        type|year|region|
+---+----------+------------+------------+-------+---------+-----+----------+----------+----------+-----------+------------+----+------+
|  0|2015-12-27|        1.33|    64236.62|1036.74| 54454.85|48.16|   8696.87|   8603.62|     93.25|        0.0|conventional|2015|Albany|
|  1|2015-12-20|        1.35|    54876.98| 674.28| 44638.81|58.33|   9505.56|   9408.07|     97.49|        0.0|conventional|2015|Albany|
|  2|2015-12-13|        0.93|   118220.22|  794.7|109149.67|130.5|   8145.35|   8042.21|    103.14|        0.0|conventional|2015|Albany|
|  3|2015-12-06|        1.08|    78992.15| 1132.0| 71976.41|72.58|   5811.16|    5677.4|    133.76|        0.0|conventional|2015|Albany|
|  4|2015-11-29|        1.28|     51039.6

Filtering

In [ ]:
df_avocado.where("year = '2015'").show()

+---+----------+------------+------------+-------+---------+------+----------+----------+----------+-----------+------------+----+------+
|_c0|      Date|AveragePrice|Total Volume|   4046|     4225|  4770|Total Bags|Small Bags|Large Bags|XLarge Bags|        type|year|region|
+---+----------+------------+------------+-------+---------+------+----------+----------+----------+-----------+------------+----+------+
|  0|2015-12-27|        1.33|    64236.62|1036.74| 54454.85| 48.16|   8696.87|   8603.62|     93.25|        0.0|conventional|2015|Albany|
|  1|2015-12-20|        1.35|    54876.98| 674.28| 44638.81| 58.33|   9505.56|   9408.07|     97.49|        0.0|conventional|2015|Albany|
|  2|2015-12-13|        0.93|   118220.22|  794.7|109149.67| 130.5|   8145.35|   8042.21|    103.14|        0.0|conventional|2015|Albany|
|  3|2015-12-06|        1.08|    78992.15| 1132.0| 71976.41| 72.58|   5811.16|    5677.4|    133.76|        0.0|conventional|2015|Albany|
|  4|2015-11-29|        1.28|     

In [ ]:
df_avocado.where("`Total Volume` < 500").show(n=df_avocado.count(), truncate=False)



+---+----------+------------+------------+----+-----+----+----------+----------+----------+-----------+-------+----+-----------------+
|_c0|Date      |AveragePrice|Total Volume|4046|4225 |4770|Total Bags|Small Bags|Large Bags|XLarge Bags|type   |year|region           |
+---+----------+------------+------------+----+-----+----+----------+----------+----------+-----------+-------+----+-----------------+
|51 |2015-01-04|1.73        |379.82      |0.0 |59.82|0.0 |320.0     |320.0     |0.0       |0.0        |organic|2015|BuffaloRochester |
|7  |2015-11-08|1.59        |84.56       |3.95|3.95 |0.0 |76.66     |73.33     |3.33      |0.0        |organic|2015|MiamiFtLauderdale|
|23 |2015-07-19|2.07        |482.26      |0.0 |75.74|0.0 |406.52    |406.52    |0.0       |0.0        |organic|2015|Syracuse         |
|24 |2015-07-12|2.05        |419.98      |0.0 |63.42|0.0 |356.56    |356.56    |0.0       |0.0        |organic|2015|Syracuse         |
|8  |2016-10-30|1.58        |385.55      |8.13|47.42|0.

In [ ]:
df_avocado.where("AveragePrice > '3'").show()

+---+----------+------------+------------+-------+--------+------+----------+----------+----------+-----------+-------+----+-----------------+
|_c0|      Date|AveragePrice|Total Volume|   4046|    4225|  4770|Total Bags|Small Bags|Large Bags|XLarge Bags|   type|year|           region|
+---+----------+------------+------------+-------+--------+------+----------+----------+----------+-----------+-------+----+-----------------+
| 12|2016-10-02|        3.03|     3714.71| 296.71|  2699.8|   0.0|     718.2|     718.2|       0.0|        0.0|organic|2016|         LasVegas|
|  7|2016-11-06|        3.12|     19043.8|5898.49|10039.34|   0.0|   3105.97|    3079.3|     26.67|        0.0|organic|2016|     SanFrancisco|
|  8|2016-10-30|        3.25|    16700.94|2325.93|11142.85|   0.0|   3232.16|   3232.16|       0.0|        0.0|organic|2016|     SanFrancisco|
| 42|2017-03-12|        3.05|     2068.26|1043.83|   77.36|   0.0|    947.07|    926.67|      20.4|        0.0|organic|2017|MiamiFtLauderdale|

In [ ]:
df_filtered = df_avocado.filter((df_avocado["Type"] == "conventional") & (df_avocado["AveragePrice"] < 0.50))
df_filtered.show()

+---+----------+------------+------------+----------+---------+--------+----------+----------+----------+-----------+------------+----+-------------+
|_c0|      Date|AveragePrice|Total Volume|      4046|     4225|    4770|Total Bags|Small Bags|Large Bags|XLarge Bags|        type|year|       region|
+---+----------+------------+------------+----------+---------+--------+----------+----------+----------+-----------+------------+----+-------------+
|  0|2015-12-27|        0.49|  1137707.43|  738314.8|286858.37|11642.46|  100891.8|  70749.02|  30142.78|        0.0|conventional|2015|PhoenixTucson|
| 47|2017-02-05|        0.46|  2200550.27|1200632.86|531226.65|18324.93| 450365.83| 113752.17|  330583.1|    6030.56|conventional|2017|PhoenixTucson|
+---+----------+------------+------------+----------+---------+--------+----------+----------+----------+-----------+------------+----+-------------+



In [ ]:
date_threshold = "2015-12-27"
volume_threshold = 50000

df_filtered_regions = df_avocado.filter((col("Date") >= date_threshold) & (df_avocado["`Total Volume`"] > volume_threshold))
df_filtered_regions.show()

+---+----------+------------+------------+----------+----------+---------+----------+----------+----------+-----------+------------+----+-------------------+
|_c0|      Date|AveragePrice|Total Volume|      4046|      4225|     4770|Total Bags|Small Bags|Large Bags|XLarge Bags|        type|year|             region|
+---+----------+------------+------------+----------+----------+---------+----------+----------+----------+-----------+------------+----+-------------------+
|  0|2015-12-27|        1.33|    64236.62|   1036.74|  54454.85|    48.16|   8696.87|   8603.62|     93.25|        0.0|conventional|2015|             Albany|
|  0|2015-12-27|        0.99|   386100.49| 292097.36|  27350.92|    297.9|  66354.31|  48605.95|  17748.36|        0.0|conventional|2015|            Atlanta|
|  0|2015-12-27|        1.17|    596819.4|  40450.49| 394104.02| 17353.79|  144911.1| 142543.88|   2367.22|        0.0|conventional|2015|BaltimoreWashington|
|  0|2015-12-27|        0.97|    62909.69|  30482.25

In [ ]:
#Tutte le regioni presenti
df_avocado.select("region").distinct().show(df_avocado.select("region").distinct().count(),truncate=False)

+-------------------+
|region             |
+-------------------+
|PhoenixTucson      |
|GrandRapids        |
|SouthCarolina      |
|TotalUS            |
|WestTexNewMexico   |
|Louisville         |
|Philadelphia       |
|Sacramento         |
|DallasFtWorth      |
|Indianapolis       |
|LasVegas           |
|Nashville          |
|GreatLakes         |
|Detroit            |
|Albany             |
|Portland           |
|CincinnatiDayton   |
|SanDiego           |
|Boise              |
|HarrisburgScranton |
|StLouis            |
|NewOrleansMobile   |
|Columbus           |
|Pittsburgh         |
|MiamiFtLauderdale  |
|SouthCentral       |
|Chicago            |
|BuffaloRochester   |
|Tampa              |
|Southeast          |
|Plains             |
|Atlanta            |
|BaltimoreWashington|
|Seattle            |
|SanFrancisco       |
|HartfordSpringfield|
|Spokane            |
|NorthernNewEngland |
|Roanoke            |
|LosAngeles         |
|Houston            |
|Jacksonville       |
|RaleighGr

Summarization

In [ ]:
df_avocado.where("region = 'Boston'").count()

338

In [ ]:
#Totale vendite per type
from pyspark.sql.functions import col, sum, format_number
df_avocado1 = df_avocado.withColumn("Total Volume", col("Total Volume").cast("double"))
df_sum_volume = df_avocado1.groupBy("type").agg(sum("Total Volume").alias("Total Volume Sum"))
df_sum_volume_formatted = df_sum_volume.withColumn("Total Volume", format_number(col("Total Volume Sum"), 2))
df_selected_columns = df_sum_volume_formatted.select("type", "Total Volume")
df_selected_columns.show()

+------------+-----------------+
|        type|     Total Volume|
+------------+-----------------+
|     organic|   436,181,682.09|
|conventional|15,087,220,911.31|
+------------+-----------------+



In [ ]:
#Prezzo medio per regione ordinato per ordine decrescente
from pyspark.sql.functions import col, avg, round

df_avg_price = df_avocado.groupBy("region").avg("AveragePrice")

df_avg_price_rounded = df_avg_price.withColumn("avg(AveragePrice)", round(col("avg(AveragePrice)"), 2))
df_avg_price_sorted = df_avg_price_rounded.orderBy(col("avg(AveragePrice)").desc())
df_avg_price_sorted.show(df_avg_price_sorted.count(), truncate=False)

+-------------------+-----------------+
|region             |avg(AveragePrice)|
+-------------------+-----------------+
|HartfordSpringfield|1.82             |
|SanFrancisco       |1.8              |
|NewYork            |1.73             |
|Philadelphia       |1.63             |
|Sacramento         |1.62             |
|Charlotte          |1.61             |
|Northeast          |1.6              |
|Albany             |1.56             |
|Chicago            |1.56             |
|RaleighGreensboro  |1.56             |
|BaltimoreWashington|1.53             |
|Boston             |1.53             |
|BuffaloRochester   |1.52             |
|Syracuse           |1.52             |
|HarrisburgScranton |1.51             |
|Jacksonville       |1.51             |
|Orlando            |1.51             |
|GrandRapids        |1.5              |
|NorthernNewEngland |1.48             |
|Spokane            |1.45             |
|Plains             |1.44             |
|Seattle            |1.44             |


In [ ]:
#Regione con il prezzo medio più alto
for anno in range(2015, 2019):

    df_year = df_avocado.filter(df_avocado.year == anno)

    df_avg_price = df_year.groupBy("region").agg(avg("AveragePrice").alias("AveragePriceMean"))

    regione_con_prezzo_massimo = df_avg_price.orderBy(col("AveragePriceMean").desc()).first()["region"]

    print("Regione con il prezzo medio più alto nel", anno, ":", regione_con_prezzo_massimo)

Regione con il prezzo medio più alto nel 2015 : HartfordSpringfield
Regione con il prezzo medio più alto nel 2016 : SanFrancisco
Regione con il prezzo medio più alto nel 2017 : SanFrancisco
Regione con il prezzo medio più alto nel 2018 : HartfordSpringfield


In [ ]:
#Somma del volume totale venduto per tipo ogni anno
from pyspark.sql.functions import col, sum, round, format_number

df_total_volume_by_type_year = df_avocado.groupBy("year", "type") \
    .agg(round(sum("Total Volume"), 2).alias("TotalVolume")) \
    .orderBy("year")

df_total_volume_by_type_year_scientific = df_total_volume_by_type_year.withColumn("TotalVolume", format_number(col("TotalVolume"), 2))

df_total_volume_by_type_year_scientific.show()


+----+------------+----------------+
|year|        type|     TotalVolume|
+----+------------+----------------+
|2015|     organic|   88,869,426.35|
|2015|conventional|4,296,599,235.69|
|2016|conventional|4,690,249,813.05|
|2016|     organic|  130,640,078.78|
|2017|conventional|4,766,165,793.44|
|2017|     organic|  168,139,905.83|
|2018|conventional|1,334,206,069.13|
|2018|     organic|   48,532,271.13|
+----+------------+----------------+



In [ ]:
#Prime 10 regioni con la somma delle bags vendute
from pyspark.sql.functions import col, sum, format_number

# Raggruppa per "region" e "type" e calcola la somma delle borse per ogni tipo di borsa
df_bag_sum_by_region_type = df_avocado.groupBy("region", "type")\
    .agg(sum("Small Bags").alias("TotalSmallBags"),
         sum("Large Bags").alias("TotalLargeBags"),
         sum("XLarge Bags").alias("TotalXLargeBags"))

# Formatta le colonne con notazione scientifica
df_bag_sum_by_region_type_scientific = df_bag_sum_by_region_type.withColumn("TotalSmallBags", format_number(col("TotalSmallBags"), 2)) \
    .withColumn("TotalLargeBags", format_number(col("TotalLargeBags"), 2)) \
    .withColumn("TotalXLargeBags", format_number(col("TotalXLargeBags"), 2))

# Mostra il risultato
df_bag_sum_by_region_type_scientific.limit(10).show()


+-------------------+------------+--------------+--------------+---------------+
|             region|        type|TotalSmallBags|TotalLargeBags|TotalXLargeBags|
+-------------------+------------+--------------+--------------+---------------+
|BaltimoreWashington|conventional| 31,992,078.72|    859,957.86|     206,985.36|
|          Southeast|     organic|  4,531,751.99|  1,701,871.79|           0.00|
|              Tampa|     organic|    491,360.99|     76,000.20|           0.00|
|           SanDiego|     organic|    492,151.33|    142,167.98|           0.00|
|            Houston|conventional| 30,272,433.59| 17,253,880.87|     326,132.75|
|         California|conventional|220,811,573.71| 11,757,395.08|   5,272,577.91|
|               West|     organic|  7,103,122.08| 11,165,346.30|       3,839.93|
|       Jacksonville|conventional|  4,140,232.43|  5,176,106.15|     105,335.77|
|          Nashville|     organic|    217,694.04|    488,286.23|           0.00|
|            Orlando|     or

In [ ]:
#Raggruppa per tipo di avocado e anno e mostrare la somma delle borse di diverse dimensioni
from pyspark.sql.functions import sum, format_number

df_bag_sum_by_type_year = df_avocado.groupBy("type", "year") \
    .agg(sum("Small Bags").alias("TotalSmallBags"),
         sum("Large Bags").alias("TotalLargeBags"),
         sum("XLarge Bags").alias("TotalXLargeBags"))

df_bag_sum_by_type_year_scientific = df_bag_sum_by_type_year.withColumn("TotalSmallBags", format_number("TotalSmallBags", 2)) \
    .withColumn("TotalLargeBags", format_number("TotalLargeBags", 2)) \
    .withColumn("TotalXLargeBags", format_number("TotalXLargeBags", 2))

df_bag_sum_by_type_year_scientific_ordered = df_bag_sum_by_type_year_scientific.orderBy("type", "year")

df_bag_sum_by_type_year_scientific_ordered.show()




+------------+----+----------------+--------------+---------------+
|        type|year|  TotalSmallBags|TotalLargeBags|TotalXLargeBags|
+------------+----+----------------+--------------+---------------+
|conventional|2015|  619,488,561.56|123,169,208.60|   5,443,128.28|
|conventional|2016|1,064,425,793.77|313,637,141.94|  20,038,284.84|
|conventional|2017|1,145,835,375.25|371,365,524.48|  23,991,658.41|
|conventional|2018|  334,433,378.67|117,731,084.73|   7,205,184.07|
|     organic|2015|   15,194,143.71|  8,897,191.88|           0.00|
|     organic|2016|   42,068,445.85| 22,989,200.12|           0.00|
|     organic|2017|   77,117,149.62| 27,973,515.65|       5,513.93|
|     organic|2018|   26,307,989.08|  5,852,903.15|       5,407.80|
+------------+----+----------------+--------------+---------------+



In [ ]:
#Raggruppa per tipo di avocado, anno, regione e codice PLU, e calcola le somme
from pyspark.sql.functions import sum, format_number spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

df_sum_by_type_year_region_plu = df_avocado.groupBy("type", "year", "region")
.agg(sum("4046").alias("TotalVolume_4046"), sum("4225").alias("TotalVolume_4225"), sum("4770").alias("TotalVolume_4770"))

df_sum_by_type_year_region_plu_scientific = df_sum_by_type_year_region_plu.withColumn("TotalVolume_4046", format_number("TotalVolume_4046", 2))
.withColumn("TotalVolume_4225", format_number("TotalVolume_4225", 2))
.withColumn("TotalVolume_4770", format_number("TotalVolume_4770", 2))

df_sum_by_type_year_region_plu_scientific_ordered = df_sum_by_type_year_region_plu_scientific.orderBy("type", "year")

df_sum_by_type_year_region_plu_scientific_ordered.show(df_sum_by_type_year_region_plu_scientific_ordered.count(), truncate=False)
df_sum_by_type_year_region_plu_scientific_ordered.show()